# 모델 매개변수 최적화

데이터에 매개변수를 최적화 -> 모델 학습, 검증, 테스트 진행

In [1]:
# 이전코드 재작성
# Dataset, DataLoader, 신경망 구성 코드 가져오기

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 하이퍼파라미터 조정
- epoch : 데이터셋을 반복하는 횟수
- batch_size : 매개변수 갱신 전 신경망에 전파된 데이터 샘플 수
- learning rate : 배치, 에폭에서 모델의 매개변수를 조절하는 비율
    - 값이 작을수록 학습속도가 느리고, 값이 클수록 예측불가능한 동작 발생할 수 있음

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop (최적화 단계)
하나의 epoch는 아래 두 부분으로 구성
- train loop : 학습용 데이터셋을 반복(iterate), 최적 매개변수로 수렴
- validation/ test loop : 모델 성능이 개선되는지 확인을 위해 테스트 데이터셋 반복(iterate)

## loss function (손실함수)
손실함수의 경우 실제값, 예측값 간의 차이를 계산, 이 차이를 최소화